In [6]:
# 야후 파이낸스 설치
!pip install yfinance --upgrade --no-cache-dir

Requirement already up-to-date: yfinance in /usr/local/lib/python3.6/dist-packages (0.1.54)


In [7]:
# 모듈가져오기
import warnings
# 경고 무시 처리
warnings.filterwarnings('ignore')

# 기본
import numpy as np
import pandas as pd

# 예측용
from fbprophet import Prophet
# 시간처리
from datetime import datetime


In [8]:
# 데이터 획득 -> 종목코드를 넣어서 주식 데이터를 획득
# 야후 파이낸스에서 획득, 국내 공공데이터에서 제공받을 수 있다.
from pandas_datareader import data
import yfinance as yf
# 재정의 -> 최신상태로 데이터를 재정의
yf.pdr_override()

In [27]:
# kospi 자료 획득 https://marketdata.krx.co.kr/mdi#document=13020101
# /content/drive/My Drive/데이터분석 프로젝트/data/kospi_list.csv
#path = '/content/drive/My Drive/데이터분석 프로젝트/data/kospi_list.csv'
kospi_list = pd.read_csv('/content/drive/My Drive/데이터분석 프로젝트/data/kospi_list.csv', encoding='utf-8')
kospi_list.head(5)

,Unnamed: 0,종목코드,종목명,현재가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,시가총액비중(%),상장주식수
0,NaN,000020,동화약품,"21,650","4,900",29.2,"16,950","21,750","16,450","10,002,002","199,410,486,900","604,716,325,500",0.04,"27,931,470"
1,NaN,000040,KR모터스,750,-13,-1.7,758,768,734,"770,766","577,433,544","68,745,763,500",0.00,"91,661,018"
2,NaN,000050,경방,"11,200",-200,-1.8,"11,400","11,450","11,200","24,419","275,168,550","307,051,024,000",0.02,"27,415,270"
3,NaN,000060,메리츠화재,"13,200",350,2.7,"12,850","13,350","12,800","266,771","3,521,255,750","1,500,576,000,000",0.10,"113,680,000"
4,NaN,000070,삼양홀딩스,"71,700","-12,200",-14.5,"80,200","81,700","71,700","346,222","26,373,349,900","614,058,230,700",0.04,"8,564,271"


In [28]:
stock_name = kospi_list['종목명']
stock_name

0          동화약품
1         KR모터스
2            경방
3         메리츠화재
4         삼양홀딩스
         ...   
903     NH프라임리츠
904     케이씨씨글라스
905        대덕전자
906      대덕전자1우
907    엘브이엠씨홀딩스
Name: 종목명, Length: 908, dtype: object

In [29]:
stock_code = kospi_list['종목코드']
stock_code

0      000020
1      000040
2      000050
3      000060
4      000070
        ...  
903    338100
904    344820
905    353200
906    35320K
907    900140
Name: 종목코드, Length: 908, dtype: object

In [30]:
kospi_list.shape

(908, 14)

In [31]:
for i in kospi_list['종목코드']:
  print(i)
  break

000020


In [32]:
# list에 담기는지 실험 해보는 코드
start_date = '2020-02-09'#'2010-01-01'
end_date = '2020-07-22'
tmp = list()
tmp.append(data.get_data_yahoo( '000020.ks', start_date, end_date))
tmp.append(data.get_data_yahoo( '000040.ks', start_date, end_date))
len(tmp)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


2

In [33]:
tmp[0]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-10,7680.0,7690.0,7490.0,7620.0,7620.0,120898
2020-02-11,7650.0,7680.0,7570.0,7650.0,7650.0,66183
2020-02-12,7650.0,7690.0,7540.0,7630.0,7630.0,95648
2020-02-13,7660.0,7670.0,7540.0,7580.0,7580.0,98128
2020-02-14,7600.0,7700.0,7550.0,7590.0,7590.0,100164
...,...,...,...,...,...,...
2020-07-16,15750.0,15850.0,15050.0,15200.0,15200.0,728717
2020-07-17,15050.0,16050.0,15050.0,15550.0,15550.0,708937
2020-07-20,15750.0,16400.0,15650.0,16300.0,16300.0,847845


In [34]:
# 배열안에 df를 저장
from tqdm.notebook import tqdm
start_date = '2020-02-14'#'2010-01-01'
end_date = '2020-07-22'
stock_list = list()
for kospi in tqdm(kospi_list['종목코드']):
  stock_list.append(data.get_data_yahoo( f'{kospi}.ks', start_date, end_date))



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [35]:
len(stock_list)

908

In [36]:
stock_list[0].index.values

array(['2020-02-14T00:00:00.000000000', '2020-02-17T00:00:00.000000000',
       '2020-02-18T00:00:00.000000000', '2020-02-19T00:00:00.000000000',
       '2020-02-20T00:00:00.000000000', '2020-02-21T00:00:00.000000000',
       '2020-02-24T00:00:00.000000000', '2020-02-25T00:00:00.000000000',
       '2020-02-26T00:00:00.000000000', '2020-02-27T00:00:00.000000000',
       '2020-02-28T00:00:00.000000000', '2020-03-02T00:00:00.000000000',
       '2020-03-03T00:00:00.000000000', '2020-03-04T00:00:00.000000000',
       '2020-03-05T00:00:00.000000000', '2020-03-06T00:00:00.000000000',
       '2020-03-09T00:00:00.000000000', '2020-03-10T00:00:00.000000000',
       '2020-03-11T00:00:00.000000000', '2020-03-12T00:00:00.000000000',
       '2020-03-13T00:00:00.000000000', '2020-03-16T00:00:00.000000000',
       '2020-03-17T00:00:00.000000000', '2020-03-18T00:00:00.000000000',
       '2020-03-19T00:00:00.000000000', '2020-03-20T00:00:00.000000000',
       '2020-03-23T00:00:00.000000000', '2020-03-24

In [37]:
stock_list[0]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-02-14,7600.0,7700.0,7550.0,7590.0,7590.0,100164
2020-02-17,7600.0,7610.0,7510.0,7550.0,7550.0,53442
2020-02-18,7520.0,7670.0,7380.0,7390.0,7390.0,256144
2020-02-19,7420.0,7480.0,7390.0,7450.0,7450.0,43486
2020-02-20,7520.0,7540.0,7260.0,7330.0,7330.0,139692
...,...,...,...,...,...,...
2020-07-16,15750.0,15850.0,15050.0,15200.0,15200.0,728717
2020-07-17,15050.0,16050.0,15050.0,15550.0,15550.0,708937
2020-07-20,15750.0,16400.0,15650.0,16300.0,16300.0,847845


In [38]:
tmp = list()
for idx, stock in enumerate(stock_list):
  tmp.append(stock[['Adj Close']])


In [39]:
len(tmp)

908

In [40]:
stock_name[0]

'동화약품'

In [41]:
for idx, stock in enumerate(stock_list):
  tmp[idx]['name'] = stock_name[idx]
  tmp[idx]['code'] = stock_code[idx]

tmp[0]

,Adj Close,name,code
Date,,,
2020-02-14,7590.0,동화약품,000020
2020-02-17,7550.0,동화약품,000020
2020-02-18,7390.0,동화약품,000020
2020-02-19,7450.0,동화약품,000020
2020-02-20,7330.0,동화약품,000020
...,...,...,...
2020-07-16,15200.0,동화약품,000020
2020-07-17,15550.0,동화약품,000020
2020-07-20,16300.0,동화약품,000020


In [42]:
tmp[1]

,Adj Close,name,code
Date,,,
2020-02-14,1000.0,KR모터스,000040
2020-02-17,1000.0,KR모터스,000040
2020-02-18,1000.0,KR모터스,000040
2020-02-19,925.0,KR모터스,000040
2020-02-20,790.0,KR모터스,000040
...,...,...,...
2020-07-16,798.0,KR모터스,000040
2020-07-17,796.0,KR모터스,000040
2020-07-20,783.0,KR모터스,000040


In [43]:
len(tmp), tmp[0].dtypes

(908, Adj Close    float64
 name          object
 code          object
 dtype: object)

In [44]:
for idx in range(len(tmp)):
  print(tmp[idx].loc[ '2020-02-14', 'Adj Close'])


7590.0
1000.0
9500.0
15250.0
56000.0
40000.0
29500.0
17150.0
221500.0
207000.0
143500.0
13950.0
16050.0
63700.0
57600.0
57000.0
1550.0
80800.0
37550.0
11400.0
7490.0
19500.0
9960.0
12950.0
41650.0
774.0
11600.0
20550.0
2420.0
4960.0
1745.0
3500.0
79000.0
6120.0
21100.0
20200.0
2850.0
5600.0
20050.0
57000.0
93300.0
64000.0
104500.0
634000.0
2395.0
6530.0
38950.0
121500.0
14350.0
211500.0
148000.0
39850.0
18650.0
22900.0
30250.0
13000.0
1055.0
3670.0
14250.0
10000.0
30150.0
51400.0
552.0
95500.0
47200.0


KeyError: ignored

In [ ]:
#  7월 22일 기준 2월 14일 종가에 미치지 못하는 종목들
less_list = list()
for idx in range(len(tmp)):
  try:
    if tmp[idx].loc[ '2020-02-14', 'Adj Close']> tmp[idx].loc[ '2020-07-22', 'Adj Close']:
      less_list.append(tmp[idx])
  except KeyError:
    print("에러가난 인덱스는 : ", idx)


print("저장된 데이터 수", len(less_list))


In [ ]:
for idx in range(len(less_list)):
  print(less_list[idx]['name'].unique())

In [ ]:
#  7월 22일 기준 3월 13일 종가에 미치지 못하는 종목들
less_list_0314 = list()
for idx in range(len(tmp)):
  try:
    if tmp[idx].loc[ '2020-03-13', 'Adj Close']> tmp[idx].loc[ '2020-07-22', 'Adj Close']:
      less_list_0314.append(tmp[idx])
  except KeyError:
    print("에러가난 인덱스는 : ", idx)


print("저장된 데이터 수", len(less_list_0314))


In [ ]:
for idx in range(len(less_list_0314)):
  print(less_list_0314[idx]['name'].unique())

In [ ]:
#  7월 22일 기준 3월 20일 종가에 미치지 못하는 종목들
less_list_0320 = list()
for idx in range(len(less_list_0314)):
  try:
    if less_list_0314[idx].loc[ '2020-03-20', 'Adj Close']> less_list_0314[idx].loc[ '2020-07-22', 'Adj Close']:
      less_list_0320.append(less_list_0314[idx])
  except KeyError:
    print("에러가난 인덱스는 : ", idx)


print("저장된 데이터 수", len(less_list_0320))

In [ ]:
for idx in range(len(less_list_0320)):
  print(less_list_0320[idx]['name'].unique())

In [ ]:
tmp[0].head()

In [ ]:
#  3월 20일 대폭락 이후 7월22일 기준 회복량이 가장 높은 KOSPI종목 Top5
less_list = list()
max_val = 0
save_idx = list()
save_per = list()
for idx in range(len(tmp)):
  try:
    a = tmp[idx].loc['2020-03-20', 'Adj Close']
    b = tmp[idx].loc['2020-07-22', 'Adj Close']
    per = (b-a)/a*100
    save_idx.append(idx)
    save_per.append(per)
  except KeyError:
    print("에러가난 인덱스는 : ", idx)


print(save_idx)


In [ ]:
per_df = pd.DataFrame(save_per, index = save_idx)
per_df

In [ ]:
per_df.sort_values(by=0, ascending=False, inplace=True)

In [ ]:
per_df.head(10)

In [ ]:
# 가장 큰 회복량을 보여준 종목의 index 는  1위 421(삼성중공 우), 2위 538(신풍제약), 3위 897(두산퓨얼셀), 4위 275(녹십자홀딩스2우), 5위 353(일양약품우)  5종목이다
tmp[353]

In [ ]:
plt.figure()
#plt.plot(tmp[421].index, tmp[421]['Adj Close'])
#plt.plot(tmp[538].index, tmp[538]['Adj Close'])
#plt.plot(tmp[897].index, tmp[897]['Adj Close'])
#plt.plot(tmp[275].index, tmp[275]['Adj Close'])
#plt.plot(tmp[353].index, tmp[353]['Adj Close'])
tmp[421].plot()
tmp[538].plot()
tmp[897].plot()
tmp[275].plot()
tmp[353].plot()
plt.show()

In [ ]:
# 3월 20일 종가를 기준으로 7월 22일 종가를 비교하였을때 가장 높은 회복량을 보여준 탑 5개 종목의 차트를 분석해보니
# 공통점은 3월 20일 대폭락장 이전과 이후에 가격의 변동이 없다가 호재의 영향으로만 높은 회복율을 띄었다.
# 다음 순위를 살펴보자
# 하락장 이후 회복한 주식의 순위는
# 1. 708(일진다이아)
# 2. 577(디피씨)
# 3. 884(효성중공업)
# 4. 67(중외제약)
# 5. 808(코오롱멀티리얼)
# 6. 47(현대건설우)
# 7. 647(유니온머티리얼)
# 8. 15(두산2우B)
# 9. 279(모나미)
# 10. 834(드림텍)
per_df[19:40]

In [ ]:
tmp[834]

In [ ]:
# 1. 708(일진다이아)
# 2. 577(디피씨)
# 3. 884(효성중공업)
# 4. 67(중외제약)
# 5. 808(코오롱멀티리얼)
# 6. 47(현대건설우)
# 7. 647(유니온머티리얼)
# 8. 15(두산2우B)
# 9. 279(모나미)
# 10. 834(드림텍)

tmp[708].plot()
tmp[577].plot()
tmp[884].plot()
tmp[67].plot()
tmp[808].plot()
tmp[47].plot()
tmp[647].plot()
tmp[15].plot()
tmp[279].plot()
tmp[834].plot()

In [ ]:
tmp[0].shape

In [ ]:
# 독립변수 환율 데이터 /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/exchange_rate.csv
path1 = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/exchange_rate.csv'
usd_df = pd.read_csv(path1, encoding='utf-8')
usd_df.shape


In [ ]:
usd_df['Date'].values, tmp[708].index.values

In [ ]:
tmp[1]['Date'] = tmp[1].index
tmp[1]['Date'].values

In [ ]:
tmp[0]['Date'] = tmp[0].index

In [ ]:
tmp[0]

In [ ]:
tmp[0]['Date'] = tmp[0]['Date'].astype('object')
tmp[0].dtypes

In [ ]:
tmp[0]['Date'].shape

In [ ]:
for i in tmp:
  i['Date'] = i.index 

In [ ]:
date_list = list()
for i in range(110):
  if tmp[0]['Date'].values[i].day<10:
    date_list.append(f"{tmp[0]['Date'].values[i].year}-0{tmp[0]['Date'].values[i].month}-0{tmp[0]['Date'].values[i].day}")
  else:
    date_list.append(f"{tmp[0]['Date'].values[i].year}-0{tmp[0]['Date'].values[i].month}-{tmp[0]['Date'].values[i].day}")
  

print(date_list)

In [ ]:
tmp[0].shape

In [ ]:
for i in tmp:
  try:
    i['Date'] = date_list
  except Exception as e:
    print(e)
  

In [ ]:
for i in tmp:
  i.set_index(['Date'], inplace=True)

In [ ]:
tmp[708].index.values

In [ ]:
usd_df.set_index(['Date'], inplace=True)
usd_df.index.values

In [ ]:
tmp[0]['Date'].apply(lambda x : x[:9])

In [ ]:
plt.plot(tmp[708].index, tmp[708]['Adj Close'])

In [ ]:
plt.plot(usd_df.index, usd_df['Exchange'])

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(usd_df.index, usd_df['Exchange'])
plt.plot(tmp[708].index, tmp[708]['Adj Close']/1200)
plt.xticks(rotation=90)
plt.show()

In [ ]:
usd_df

In [ ]:
for i in tmp:
  i['dollar'] = usd_df['Exchange']

In [ ]:
tmp[0].head()

In [ ]:
sm.add_constant(tmp[708]['dollar'])

In [ ]:
import statsmodels.api as sm
import seaborn as sns
model = sm.OLS(tmp[708]['Adj Close'], sm.add_constant(tmp[708]['dollar']))

In [ ]:
result = model.fit()

In [ ]:
print(result.summary())

In [ ]:
result.rsquared, result.rsquared_adj, result.predict(), result.aic

In [ ]:
a, b = result.params

In [ ]:
# 시각화
# 원 데이터를 산포도로 시각화
plt.scatter(tmp[708]['dollar'], tmp[708]['Adj Close'], label='predict')

# 학습된 형태로 선형 회귀 결과를 드로잉
# x대비 방정식에 따라 y를 드로잉 -> 선형 차트
# 방정식으로 표현
# y = a + b*x
plt.plot(tmp[708]['dollar'], a+(tmp[708]['dollar']*b), label='reg output')
# 0.1은 선이 겹쳐서 편향을 부여했음
# 방정식을 통해 나온 예측값으로 표현 == 방정식으로 계산
plt.plot(tmp[708]['dollar'], 0.1+result.predict(), label='reg output')

plt.xlabel('dollar')
plt.ylabel('Adj Close')
plt.legend()
plt.show()

In [ ]:
##############################################################
# 유가 데이터 로드 /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/oil.csv
path2 = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/oil.csv'
oil_df = pd.read_csv(path2, encoding='utf-8')
oil_df.set_index(['Date'], inplace=True)
oil_df

In [ ]:
# 결측치 -를 NaN으로 처리
oil_df['Dubai'] = oil_df['Dubai'].apply(lambda x : x.replace('-', np.NaN))
oil_df['WTI'] = oil_df['WTI'].apply(lambda x : x.replace('-', np.NaN))
oil_df['Oman'] = oil_df['Oman'].apply(lambda x : x.replace('-', np.NaN))

In [ ]:
oil_df.WTI.values

In [ ]:
oil_df[oil_df['Dubai']=='-']=np.nan
oil_df[oil_df['WTI']=='-']=np.nan
oil_df[oil_df['Oman']=='-']=np.nan

In [ ]:
oil_df.columns

In [ ]:
oil_df[oil_df['Brent']=='nan']=np.nan
oil_df[oil_df['WTI']=='nan']=np.nan
oil_df[oil_df['Oman']=='nan']=np.nan

In [ ]:
# 결측치 채움
oil_df.fillna(method='pad', inplace=True)
oil_df.WTI.values

In [ ]:
oil_df.shape

In [ ]:
oil_df['WTI'].values

In [ ]:
#oil_df = oil_df.drop(['2020-02-17', '2020-07-03', '2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05'])
oil_df = oil_df.drop(['2020-05-25'])
oil_df.shape

In [ ]:
test.drop(['2020-02-17', '2020-04-10', '2020-07-03'])

In [ ]:
for i in range(len(tmp)):
  try:
    #tmp[i] = tmp[i].drop(['2020-02-17', '2020-04-10', '2020-07-03'])
    tmp[i] = tmp[i].drop(['2020-05-25'])
  except Exception as e:
    print(e)
  

In [ ]:
# 1. 708(일진다이아)
# 2. 577(디피씨)
# 3. 884(효성중공업)
# 4. 67(중외제약)
# 5. 808(코오롱멀티리얼)
# 6. 47(현대건설우)
# 7. 647(유니온머티리얼)
# 8. 15(두산2우B)
# 9. 279(모나미)
# 10. 834(드림텍)
tmp[708].shape, tmp[577].shape, tmp[884].shape, tmp[67].shape, tmp[808].shape, tmp[47].shape, tmp[647].shape, tmp[15].shape, tmp[279].shape, tmp[834].shape

In [ ]:
stock_top10 = list()
stock_top10.append(tmp[708])
stock_top10.append(tmp[577])
stock_top10.append(tmp[884])
stock_top10.append(tmp[67])
stock_top10.append(tmp[808])
stock_top10.append(tmp[47])
stock_top10.append(tmp[647])
stock_top10.append(tmp[15])
stock_top10.append(tmp[279])
stock_top10.append(tmp[834])

In [ ]:
for i in range(len(stock_top10))

In [ ]:
# usd_df.drop(['2020-02-17', '2020-04-10', '2020-07-03'], inplace=True)
usd_df.drop(['2020-05-25'], inplace=True)

In [ ]:
oil_df.columns

In [ ]:
oil_df.Dubai.values

In [ ]:
oil_df

In [ ]:
oil_df['Dubai']

In [ ]:

for i in range(len(stock_top10)):
  stock_top10[i]['dollar'] = usd_df['Exchange']
  stock_top10[i]['Dubai'] = oil_df['Dubai']
  stock_top10[i]['Brent'] = oil_df['Brent']
  stock_top10[i]['Oman'] = oil_df['Oman']
  stock_top10[i]['WTI'] = oil_df['WTI']

In [ ]:
for i in range(len(stock_top10)):
  stock_list[i]['Dubai'] = oil_df['Dubai']

In [ ]:
stock_top10[0]

In [ ]:
cols = ['Adj Close', 'dollar',	'Dubai',	'Brent',	'Oman',	'WTI']
cols

In [ ]:
import seaborn as sns
sns.set(style='whitegrid')
sns.pairplot(stock_top10[0][cols], height=2.5)
plt.show()

In [ ]:
corr_tmp = np.corrcoef(stock_top10[0][cols].values)
corr_tmp.shape

In [ ]:
corr_tmp = np.corrcoef(stock_top10[0][cols].values.T)
corr_tmp.shape

In [ ]:
sns.heatmap(corr_tmp, cbar=True, annot=True, square=True, fmt='.2f',
            annot_kws={'size':15}, yticklabels=cols, xticklabels=cols)
plt.show()

In [ ]:
stock_top10[0].columns

In [ ]:
columns = stock_top10[0].columns[3:]
columns

In [ ]:
target_df = stock_top10[0][columns]
target_df

In [ ]:
y_data = stock_top10[0]['Adj Close']
y_data.head(2)

In [ ]:
from sklearn.model_selection import train_test_split # 데이터를 훈련용, 테스트용으로 분리
from sklearn.metrics import r2_score       
X_train, X_test, y_train, y_test = train_test_split(target_df, y_data, train_size=0.7, 
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train_added = sm.add_constant(X_train)
X_train_added

In [ ]:
full_model= sm.OLS(y_train, X_train_added)
full_model_res = full_model.fit()

In [ ]:
print(full_model_res.summary())

In [ ]:
print(f'{"제거된컬럼":20} {"R제곱값":10} {"다중공선성":10}')
print('-'*60)
# 순환하면서 컬럼을 10개씩 구성한다.
for idx, col in enumerate(columns):
  # 독립변수 원본을 리스트로 생성 -> 사본
  tmp3 = list(columns)
  # 반복적으로 해당 위치에 해당되는 컬럼 제거
  rm_col = tmp3.pop(idx) # 제거된 컬럼명 rm_col에 수렴
  # 다중 회귀 모델 생성
  m_model = sm.OLS( y_train, sm.add_constant(X_train[tmp3]))
  # 학습
  m_model_res = m_model.fit()
  # 다중공선성을 점검하는 지표
  vif = 1/(1-m_model_res.rsquared)
  print(f'{rm_col:20} {m_model_res.rsquared:10} {vif}')

In [ ]:
concat_top10 = pd.concat([stock_top10[0],stock_top10[1], stock_top10[2],stock_top10[3], stock_top10[4],stock_top10[5], stock_top10[6],stock_top10[7], stock_top10[8],stock_top10[9]])

In [ ]:
# kospi종목 회복량 탑 10 저장
concat_top10.to_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv', encoding='utf-8')

In [46]:
# kospi종목 회복량 탑10 데이터 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv
path = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/stock_top10.csv'
concat_top10 = pd.read_csv(path, encoding='utf-8')

In [47]:
concat_top10.shape

(1060, 9)

In [52]:
stock_top10 = list()
for i in range(10):
  stock_top10.append(concat_top10[i*106:(i+1)*106])

In [54]:
stock_top10[0]

,Date,Adj Close,name,code,dollar,Dubai,Brent,Oman,WTI
0,2020-02-14,28500.0,일진다이아,81000,1181.5,55.23,57.32,55.73,52.05
1,2020-02-18,28300.0,일진다이아,81000,1182.2,55.11,57.75,55.56,52.05
2,2020-02-19,27950.0,일진다이아,81000,1188.1,56.39,59.12,56.87,53.29
3,2020-02-20,29350.0,일진다이아,81000,1190.8,56.92,59.31,57.52,53.78
4,2020-02-21,28400.0,일진다이아,81000,1197.0,56.41,58.50,57.30,53.38
...,...,...,...,...,...,...,...,...,...
101,2020-07-16,57200.0,일진다이아,81000,1200.7,43.37,43.37,43.63,40.75
102,2020-07-17,56600.0,일진다이아,81000,1204.1,42.86,43.14,43.16,40.59
103,2020-07-20,59000.0,일진다이아,81000,1205.7,42.72,43.28,42.87,40.81
104,2020-07-21,59300.0,일진다이아,81000,1203.7,43.87,44.32,44.02,41.96


In [58]:
for i in range(10):
  del stock_top10[i]['Dubai']
  del stock_top10[i]['Brent']
  del stock_top10[i]['Oman']


KeyError: ignored

In [59]:
stock_top10[0].head()

,Date,Adj Close,name,code,dollar,WTI
0,2020-02-14,28500.0,일진다이아,81000,1181.5,52.05
1,2020-02-18,28300.0,일진다이아,81000,1182.2,52.05
2,2020-02-19,27950.0,일진다이아,81000,1188.1,53.29
3,2020-02-20,29350.0,일진다이아,81000,1190.8,53.78
4,2020-02-21,28400.0,일진다이아,81000,1197.0,53.38


In [55]:
stock_top10[0].dtypes

Date          object
Adj Close    float64
name          object
code           int64
dollar       float64
Dubai        float64
Brent        float64
Oman         float64
WTI          float64
dtype: object

In [ ]:
for i in range(len(stock_top10)):
  stock_top10[i]['Dubai'] = stock_top10[i]['Dubai'].apply(lambda x : float(x))
  stock_top10[i]['Oman'] = stock_top10[i]['Oman'].apply(lambda x : float(x))
  stock_top10[i]['WTI'] = stock_top10[i]['WTI'].apply(lambda x : float(x))

In [ ]:
oil_df.index.values

In [ ]:
tmp[0].index.values

In [ ]:
test = tmp[0]
test

In [ ]:
test.drop( )

In [ ]:
|less_list[0].index.values

In [ ]:
less_list[0].index = less_list[0].index.astype('object')

In [ ]:
less_list[0]['Date'] = less_list[0].index
less_list[0].head()

In [ ]:
less_list[0]['Date'].values

In [ ]:
less_list[0]['Date'].astype('object')

In [ ]:
less_list[0]['Date'] = less_list[0]['Date'].apply(lambda x : x.replace('T00:00:00.000000000', ''))

In [ ]:
less_list[0].index

In [ ]:
for i in less_list:
  i.index = i.index.apply(lambda x : x.replace('T00:00:00.000000000', ''))

In [ ]:
# 코스피 지수 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/kopi_rate.csv
kospi_rate = pd.read_csv('/content/drive/My Drive/데이터분석 프로젝트/data/kopi_rate.csv', encoding='utf-8')
kospi_rate

In [ ]:
kospi_rate.columns = ['Date',	'종가',	'대비',	'등락률(%)',	'시가',	'고가',	'저가',	'거래량(천주)',	'거래대금(원)',	'상장시가총액(원)']
kospi_rate.set_index( ['Date'], inplace=True)

In [ ]:
kospi_rate = kospi_rate[['종가']]
kospi_rate.dtypes

In [ ]:
kospi_rate['종가'] = kospi_rate['종가'].apply(lambda x : float(x.replace(',', '')))
kospi_rate

In [ ]:
kospi_rate.sort_index(ascending=True, inplace=True)

In [ ]:
tmp1 = kospi_rate.index
kospi_rate['Date'] = tmp1
kospi_rate['Date'] = kospi_rate['Date'].apply(lambda x:x.replace('/', '-'))
kospi_rate.set_index(['Date'], inplace=True)

In [ ]:
kospi_rate

In [ ]:
tmp[0].index


In [ ]:
tmp[0].index = tmp[0].index.apply(lambda x:object(x))

In [ ]:
tmp[1]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure()
plt.plot(kospi_rate.index, kospi_rate['종가'])
plt.show()
plt.plot(tmp[0].index.to_pydatetime(), tmp[0]['Adj Close'])
#kospi_rate.plot(kind='line')
#tmp[0].plot(kind='line')
#tmp[1].plot(kind='line')
#tmp[2].plot(kind='line')


plt.show()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [60]:
#####################################################################################
# 나스닥 데이터 불러오기 /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/ixic.csv
path3 = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/ixic.csv'
nasdaq = pd.read_csv(path3, encoding='utf-8')
nasdaq

,Date,Close
0,2020-02-14,9731.18
1,2020-02-18,9732.74
2,2020-02-19,9817.18
3,2020-02-20,9750.97
4,2020-02-21,9576.59
...,...,...
105,2020-07-16,10473.83
106,2020-07-17,10503.19
107,2020-07-20,10767.09
108,2020-07-21,10680.36


In [61]:
nasdaq.Date.values

array(['2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20',
       '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26',
       '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-03',
       '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-09',
       '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19',
       '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25',
       '2020-03-26', '2020-03-27', '2020-03-30', '2020-03-31',
       '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-06',
       '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-13',
       '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17',
       '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23',
       '2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29',
       '2020-04-30', '2020-05-01', '2020-05-04', '2020-05-05',
       '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-11',
       '2020-05-12', '2020-05-13', '2020-05-14', '2020-

In [62]:
nasdaq.set_index(['Date'], inplace=True)

In [63]:
nasdaq.drop(['2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05'])

,Close
Date,
2020-02-14,9731.18
2020-02-18,9732.74
2020-02-19,9817.18
2020-02-20,9750.97
2020-02-21,9576.59
...,...
2020-07-16,10473.83
2020-07-17,10503.19
2020-07-20,10767.09


In [73]:
nasdaq.index.values, stock_top10[0].index.values

(array(['2020-02-14', '2020-02-18', '2020-02-19', '2020-02-20',
        '2020-02-21', '2020-02-24', '2020-02-25', '2020-02-26',
        '2020-02-27', '2020-02-28', '2020-03-02', '2020-03-03',
        '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-09',
        '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
        '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19',
        '2020-03-20', '2020-03-23', '2020-03-24', '2020-03-25',
        '2020-03-26', '2020-03-27', '2020-03-30', '2020-03-31',
        '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-06',
        '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-13',
        '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17',
        '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23',
        '2020-04-24', '2020-04-27', '2020-04-28', '2020-04-29',
        '2020-04-30', '2020-05-01', '2020-05-04', '2020-05-05',
        '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-11',
        '2020-05-12', '2020-05-13', '202

In [74]:
import pandas as pd
import numpy as np


In [75]:
for i in range(10):
  stock_top10[i].set_index(['Date'], inplace=True)

In [76]:
# stock_top10 배열에 nasdaq 독립변수 넣어주기
for i in range(10): 
  stock_top10[i]['NASDAQ'] = nasdaq

In [77]:
stock_top10[2].head()

,Adj Close,name,code,dollar,WTI,NASDAQ
Date,,,,,,
2020-02-14,24400.0,효성중공업,298040,1181.5,52.05,9731.18
2020-02-18,23800.0,효성중공업,298040,1182.2,52.05,9732.74
2020-02-19,23350.0,효성중공업,298040,1188.1,53.29,9817.18
2020-02-20,22800.0,효성중공업,298040,1190.8,53.78,9750.97
2020-02-21,22250.0,효성중공업,298040,1197.0,53.38,9576.59


In [78]:
# 다우지수 데이터 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/dji.csv
path = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/dji.csv'
dow = pd.read_csv(path, encoding='utf-8')

In [79]:
dow

,Date,Close
0,2020-02-14,29398.08
1,2020-02-18,29232.19
2,2020-02-19,29348.03
3,2020-02-20,29219.98
4,2020-02-21,28992.41
...,...,...
105,2020-07-16,26734.71
106,2020-07-17,26671.95
107,2020-07-20,26680.87
108,2020-07-21,26840.40


In [80]:
dow.set_index(['Date'], inplace=True)
dow

,Close
Date,
2020-02-14,29398.08
2020-02-18,29232.19
2020-02-19,29348.03
2020-02-20,29219.98
2020-02-21,28992.41
...,...
2020-07-16,26734.71
2020-07-17,26671.95
2020-07-20,26680.87


In [81]:
dow.drop(['2020-04-15', '2020-04-30', '2020-05-01', '2020-05-05'])

,Close
Date,
2020-02-14,29398.08
2020-02-18,29232.19
2020-02-19,29348.03
2020-02-20,29219.98
2020-02-21,28992.41
...,...
2020-07-16,26734.71
2020-07-17,26671.95
2020-07-20,26680.87


In [82]:
# 금리 데이터 불러오기
# /content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/interest_rate.csv
path = '/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/interest_rate.csv'
interest_rate = pd.read_csv(path, encoding='utf-8')
interest_rate.head()

,Date,Interest
0,2020-02-14,1.24
1,2020-02-17,1.24
2,2020-02-18,1.23
3,2020-02-19,1.24
4,2020-02-20,1.23


In [86]:
interest_rate.head()

,Date,Interest
0,2020-02-14,1.24
1,2020-02-17,1.24
2,2020-02-18,1.23
3,2020-02-19,1.24
4,2020-02-20,1.23


In [87]:
interest_rate.set_index(['Date'], inplace=True)

In [88]:
interest_rate.drop(['2020-02-17', '2020-04-10', '2020-05-25','2020-07-03'])

,Interest
Date,
2020-02-14,1.24
2020-02-18,1.23
2020-02-19,1.24
2020-02-20,1.23
2020-02-21,1.23
...,...
2020-07-16,0.49
2020-07-17,0.46
2020-07-20,0.44


In [89]:
stock_top10[0].head()

,Adj Close,name,code,dollar,WTI,NASDAQ
Date,,,,,,
2020-02-14,28500.0,일진다이아,81000,1181.5,52.05,9731.18
2020-02-18,28300.0,일진다이아,81000,1182.2,52.05,9732.74
2020-02-19,27950.0,일진다이아,81000,1188.1,53.29,9817.18
2020-02-20,29350.0,일진다이아,81000,1190.8,53.78,9750.97
2020-02-21,28400.0,일진다이아,81000,1197.0,53.38,9576.59


In [91]:
for i in range(10):
  stock_top10[i]['DOW'] = dow
  stock_top10[i]['interest_rate'] = interest_rate


In [92]:
stock_top10[0].head()

,Adj Close,name,code,dollar,WTI,NASDAQ,DOW,interest_rate
Date,,,,,,,,
2020-02-14,28500.0,일진다이아,81000,1181.5,52.05,9731.18,29398.08,1.24
2020-02-18,28300.0,일진다이아,81000,1182.2,52.05,9732.74,29232.19,1.23
2020-02-19,27950.0,일진다이아,81000,1188.1,53.29,9817.18,29348.03,1.24
2020-02-20,29350.0,일진다이아,81000,1190.8,53.78,9750.97,29219.98,1.23
2020-02-21,28400.0,일진다이아,81000,1197.0,53.38,9576.59,28992.41,1.23


In [94]:
top10_variable = pd.concat([stock_top10[0], stock_top10[1], stock_top10[2], stock_top10[3], stock_top10[4], stock_top10[5], stock_top10[6], stock_top10[7], stock_top10[8], stock_top10[9]])

In [95]:
top10_variable.to_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/top10_variable.csv', encoding='utf-8')

In [100]:
variable = stock_top10[0][['dollar',	'WTI'	,'NASDAQ',	'DOW',	'interest_rate']]
variable

,dollar,WTI,NASDAQ,DOW,interest_rate
Date,,,,,
2020-02-14,1181.5,52.05,9731.18,29398.08,1.24
2020-02-18,1182.2,52.05,9732.74,29232.19,1.23
2020-02-19,1188.1,53.29,9817.18,29348.03,1.24
2020-02-20,1190.8,53.78,9750.97,29219.98,1.23
2020-02-21,1197.0,53.38,9576.59,28992.41,1.23
...,...,...,...,...,...
2020-07-16,1200.7,40.75,10473.83,26734.71,0.49
2020-07-17,1204.1,40.59,10503.19,26671.95,0.46
2020-07-20,1205.7,40.81,10767.09,26680.87,0.44


In [101]:
variable.to_csv('/content/drive/My Drive/데이터분석 프로젝트/data/독립변수 data/variable.csv', encoding='utf-8')

In [108]:
pip install pypiwin32

  Using cached https://files.pythonhosted.org/packages/d0/1b/2f292bbd742e369a100c91faa0483172cd91a1a422a6692055ac920946c5/pypiwin32-223-py3-none-any.whl
ERROR: Could not find a version that satisfies the requirement pywin32>=223 (from pypiwin32) (from versions: none)
ERROR: No matching distribution found for pywin32>=223 (from pypiwin32)


In [123]:
# open dart의 open api 이용하기
# https://opendart.fss.or.kr/mng/apiUsageStatusView.do
# 인증키 : 2fa1669db25ebdf74b7a013aca6d38acf427dcf3
auth_key = '2fa1669db25ebdf74b7a013aca6d38acf427dcf3'
# https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=2fa1669db25ebdf74b7a013aca6d38acf427dcf3
# 해당 uri를 통하여 재무제표를 구할때 필요한 인자값인 종목의 고유번호를 알아온다.
# 1. 708(일진다이아) 고유번호 : 00508274
# 2. 577(디피씨) 고유번호 : 00117498
# 3. 884(효성중공업) 고유번호 : 01316245
# 4. 67(중외제약) 고유번호 : 00149947
# 5. 808(코오롱머티리얼) 고유번호 : 00652159
# 6. 47(현대건설우) 고유번호 : 00164478(현대건설만 고유번호가 있고 현대건설우는 교유번호가 없음)
# 7. 647(유니온머티리얼) 고유번호 : 00369833
# 8. 15(두산2우B) 고유번호 : 00117212(이것도 마찬가지로 없다.)
# 9. 279(모나미) 고유번호 : 00121288
# 10. 834(드림텍) 고유번호 : 00277499

In [135]:
import urllib.request
import os
import sys
import json
# ?crtfc_key=2fa1669db25ebdf74b7a013aca6d38acf427dcf3&corp_code=00508274&bsns_year=2019&reprt_code=11011
url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
keyword = ['00508274', '00117498', '01316245', '00149947', '00652159', '00164478', '00369833', '00117212', '00121288', '00277499']
bsns_year = 2019
reprt_code = 11011
param = f'?crtfc_key={auth_key}&corp_code={keyword[0]}&bsns_year={bsns_year}&reprt_code={reprt_code}'


In [130]:
url+param

'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key=2fa1669db25ebdf74b7a013aca6d38acf427dcf3&corp_code=00508274&bsns_year=2019&reprt_code=11011'

In [131]:
target_url = url+param
target_url

'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key=2fa1669db25ebdf74b7a013aca6d38acf427dcf3&corp_code=00508274&bsns_year=2019&reprt_code=11011'

In [136]:
request = urllib.request.Request(target_url)
response = urllib.request.urlopen(request)
tmp = json.load(response)
print(tmp)

{'status': '000', 'message': '정상', 'list': [{'rcept_no': '20200330002763', 'reprt_code': '11011', 'bsns_year': '2019', 'corp_code': '00508274', 'stock_code': '081000', 'fs_div': 'CFS', 'fs_nm': '연결재무제표', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_nm': '유동자산', 'thstrm_nm': '제 16 기', 'thstrm_dt': '2019.12.31 현재', 'thstrm_amount': '175,265,993,788', 'frmtrm_nm': '제 15 기', 'frmtrm_dt': '2018.12.31 현재', 'frmtrm_amount': '96,619,575,792', 'bfefrmtrm_nm': '제 14 기', 'bfefrmtrm_dt': '2017.12.31 현재', 'bfefrmtrm_amount': '89,661,634,987', 'ord': '1'}, {'rcept_no': '20200330002763', 'reprt_code': '11011', 'bsns_year': '2019', 'corp_code': '00508274', 'stock_code': '081000', 'fs_div': 'CFS', 'fs_nm': '연결재무제표', 'sj_div': 'BS', 'sj_nm': '재무상태표', 'account_nm': '비유동자산', 'thstrm_nm': '제 16 기', 'thstrm_dt': '2019.12.31 현재', 'thstrm_amount': '73,844,391,261', 'frmtrm_nm': '제 15 기', 'frmtrm_dt': '2018.12.31 현재', 'frmtrm_amount': '63,276,153,746', 'bfefrmtrm_nm': '제 14 기', 'bfefrmtrm_dt': '2017.12.31 현재', '

In [137]:
for tmp_list in tmp['list']:
  print(tmp_list['rcept_no'])
  break

20200330002763


In [142]:
test = pd.DataFrame([tmp['list'][0]])
test

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,유동자산,제 16 기,2019.12.31 현재,"175,265,993,788",제 15 기,2018.12.31 현재,"96,619,575,792",제 14 기,2017.12.31 현재,"89,661,634,987",1


In [144]:
len(tmp['list'])

24

In [148]:
for i in range(1, len(tmp['list'])):
  test.loc[i] = tmp['list'][i]


In [149]:
test

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,thstrm_nm,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord
0,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,유동자산,제 16 기,2019.12.31 현재,"175,265,993,788",제 15 기,2018.12.31 현재,"96,619,575,792",제 14 기,2017.12.31 현재,"89,661,634,987",1
1,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,비유동자산,제 16 기,2019.12.31 현재,"73,844,391,261",제 15 기,2018.12.31 현재,"63,276,153,746",제 14 기,2017.12.31 현재,"67,273,898,397",3
2,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,자산총계,제 16 기,2019.12.31 현재,"249,110,385,049",제 15 기,2018.12.31 현재,"159,895,729,538",제 14 기,2017.12.31 현재,"156,935,533,384",5
3,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,유동부채,제 16 기,2019.12.31 현재,"48,380,555,351",제 15 기,2018.12.31 현재,"43,949,773,401",제 14 기,2017.12.31 현재,"46,310,875,346",7
4,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,비유동부채,제 16 기,2019.12.31 현재,"16,105,820,598",제 15 기,2018.12.31 현재,"12,691,867,339",제 14 기,2017.12.31 현재,"12,562,873,217",9
5,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,부채총계,제 16 기,2019.12.31 현재,"64,486,375,949",제 15 기,2018.12.31 현재,"56,641,640,740",제 14 기,2017.12.31 현재,"58,873,748,563",11
6,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,자본금,제 16 기,2019.12.31 현재,"14,202,975,000",제 15 기,2018.12.31 현재,"11,277,975,000",제 14 기,2017.12.31 현재,"11,277,975,000",13
7,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,이익잉여금,제 16 기,2019.12.31 현재,"95,749,921,792",제 15 기,2018.12.31 현재,"88,229,299,585",제 14 기,2017.12.31 현재,"82,695,181,689",17
8,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,BS,재무상태표,자본총계,제 16 기,2019.12.31 현재,"184,624,009,100",제 15 기,2018.12.31 현재,"103,254,088,797",제 14 기,2017.12.31 현재,"98,061,784,821",21
9,20200330002763,11011,2019,00508274,081000,CFS,연결재무제표,IS,손익계산서,매출액,제 16 기,2019.01.01 ~ 2019.12.31,"161,062,349,881",제 15 기,2018.01.01 ~ 2018.12.31,"133,959,367,594",제 14 기,2017.01.01 ~ 2017.12.31,"118,912,464,518",23
